In [1]:
import pandas as pd
import numpy as np

from plmdata import Backend

In [2]:
B = Backend(False)

In [18]:
# Get data we want to use for training from the backend.

df = B.get_normalized_alsfrs_series_random_users(200)

In [20]:
# Take a look

df

,user_id,report_date,score,min_date,days_since_first_report
0,1914,722815,12,722815,0
1,1914,726516,4,722815,3701
2,1914,729212,47,722815,6397
3,1914,733287,14,722815,10472
4,1914,733321,14,722815,10506
...,...,...,...,...,...
4010,1700,737335,42,737051,284
4011,1700,737352,42,737051,301
4012,1700,737385,40,737051,334
4013,1700,737454,38,737051,403


In [24]:
# How many users report a score less than 20?

users_lt20 = df[df.score <= 20].user_id.unique()

len(users_lt20)

103

In [52]:
# How many report a score less than 20 in their first five reports?
# Our dataframe is already sorted by days since first report, so
# .head() gets us what we need

first_five_by_user = df.groupby("user_id").head()

In [54]:
first_five_by_user

,user_id,report_date,score,min_date,days_since_first_report
0,1914,722815,12,722815,0
1,1914,726516,4,722815,3701
2,1914,729212,47,722815,6397
3,1914,733287,14,722815,10472
4,1914,733321,14,722815,10506
...,...,...,...,...,...
4004,1700,737051,46,737051,0
4005,1700,737124,41,737051,73
4006,1700,737158,39,737051,107
4007,1700,737212,41,737051,161


In [ ]:
# What on Earth is going on with user 1914? Use cell 118 in EDA.ipynb to
# look at that user's data - they have a somewhat normal disease progression
# starting in 2010 or so, but have stray reports in 1980, 1990, and 2000. Using 
# onset date instead of first report date # would fix this issue, but would also 
# throw out a lot of data.

In [30]:
gt20

,report_date,score,min_date,days_since_first_report
user_id,,,,
43,729975,20,729975,0
75,734746,40,734746,0
108,734985,24,734985,0
163,736073,20,736073,0
184,732609,33,732609,0
...,...,...,...,...
7233,734778,22,734778,0
7263,732585,24,732585,0
7326,729676,26,729676,0
